In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy
import seaborn as sns

### Загрузите набор данных credit-screening.txt. Выполните полную предобработку данных. Необходимо уметь объяснять каждый шаг предобработки.